## Task 4
### Solving system of linear equations with LU decomposition



In [199]:
import numpy as np

Decomposes A as the product of lower triangular L with 1 on the main diagonal and upper triangular U

LU = A

Example:
\begin{equation}
    \begin{bmatrix}
        4 & 3\\
        6 & 3
    \end{bmatrix}
    =
    \begin{bmatrix}
        \ell_{11} & 0 \\
        \ell_{21} & \ell_{22} \\
    \end{bmatrix}
    \begin{bmatrix}
        u_{11} & u_{12} \\
        0 & u_{22} \\
    \end{bmatrix}
\end{equation}

\begin{align}
  \ell_{11} \cdot u_{11} +         0 \cdot      0 &= 4 \\
  \ell_{11} \cdot u_{12} +         0 \cdot u_{22} &= 3 \\
  \ell_{21} \cdot u_{11} + \ell_{22} \cdot      0 &= 6 \\
  \ell_{21} \cdot u_{12} + \ell_{22} \cdot u_{22} &= 3.
\end{align}

\begin{align}
\ell_{11} = \ell_{22} &=  1   \\
            \ell_{21} &=  1.5 \\
               u_{11} &=  4   \\
               u_{12} &=  3   \\
               u_{22} &= -1.5
\end{align}

In [202]:
def decompose_to_lu(a_matrix):
    matrix_size = a_matrix.shape[0]
    l_matrix = np.zeros((matrix_size, matrix_size))
    u_matrix = np.zeros((matrix_size, matrix_size))

    for i in range(matrix_size):
        for k in range(i, matrix_size):
            lu_sum = sum([l_matrix[i][j] * u_matrix[j][k] for j in range(i)])
            u_matrix[i][k] = a_matrix[i][k] - lu_sum

        for k in range(i, matrix_size):
            if i == k:
                l_matrix[i][i] = 1
            else:
                lu_sum = sum([l_matrix[k][j] * u_matrix[j][i] for j in range(i)])
                l_matrix[k][i] = (a_matrix[k][i] - lu_sum) / u_matrix[i][i]

    return l_matrix, u_matrix

Solves Ax = b equation using LU decomposition

1. Solve Ly = b for y
2. Solve Ux = y for x

In [205]:
def solve_with_lu(l_matrix, u_matrix, b_vector):
    matrix_size = l_matrix.shape[0]
    y_vector = np.zeros(matrix_size)
    for i in range(matrix_size):
        y_vector[i] = b_vector[i] - sum([l_matrix[i][k] * y_vector[k] for k in range(i)])

    x_vector = np.zeros(matrix_size)
    for i in range(matrix_size - 1, -1, -1):
        x_vector[i] = (y_vector[i] - sum([u_matrix[i][k] * x_vector[k] for k in range(i + 1, matrix_size)])) \
                         / u_matrix[i][i]

    return x_vector

In [116]:
def calculate_spectral_conditionality_number(matrix):
    matrix_norm = np.linalg.norm(matrix)
    inverse_matrix = np.linalg.inv(matrix)
    inverse_matrix_norm = np.linalg.norm(inverse_matrix)

    return matrix_norm * inverse_matrix_norm

In [59]:
def get_matrix_major_minors(matrix):
    minors = []
    rows, cols = matrix.shape
    for i in range(rows):
        minor = matrix[np.arange(rows) <= i][:, np.arange(cols) <= i]
        minors.append(minor)

    return minors

def generate_matrix_with_nonzero_major_minors(size):
    matrix = np.random.rand(size, size)
    while not all(np.linalg.det(minor) != 0 for minor in get_matrix_major_minors(matrix)):
        matrix = np.random.rand(size, size)
    return matrix


### Test 1
#### Simple matrix

\begin{equation}
    \begin{bmatrix}
        1 & 2 & 3 \\
        4 & -1 & 0 \\
        -2 & 5 & 1
    \end{bmatrix}
    \begin{bmatrix}
        1 \\
        2 \\
        3
    \end{bmatrix}
    =
    \begin{bmatrix}
        14 \\
        2 \\
        11
    \end{bmatrix}
\end{equation}

\begin{align}
  \text{L:} \quad &
  \begin{bmatrix}
    1 & 0 & 0 \\
    4 & 1 & 0 \\
    -2 & -1 & 1 \\
  \end{bmatrix} \\
  \text{U:} \quad &
  \begin{bmatrix}
    1 & 2 & 3 \\
    0 & -9 & -12 \\
    0 & 0 & -5 \\
  \end{bmatrix}
\end{align}

In [206]:
a_matrix = np.array([[1, 2, 3],
                   [4, -1, 0],
                   [-2, 5, 1]])

right_side = np.array([14, 2, 11])

l_matrix, u_matrix = decompose_to_lu(a_matrix)
x_vector = solve_with_lu(l_matrix, u_matrix, right_side)

print("The answer: \n", x_vector)
print("L: \n", l_matrix)
print("U: \n", u_matrix)

print("A spectral conditionality number: ", calculate_spectral_conditionality_number(a_matrix))
print("L spectral conditionality number: ", calculate_spectral_conditionality_number(l_matrix))
print("U spectral conditionality number: ", calculate_spectral_conditionality_number(u_matrix))

The answer: 
 [1. 2. 3.]
L: 
 [[ 1.  0.  0.]
 [ 4.  1.  0.]
 [-2. -1.  1.]]
U: 
 [[  1.   2.   3.]
 [  0.  -9. -12.]
 [  0.   0.  -5.]]
A spectral conditionality number:  5.131072133050548
L spectral conditionality number:  23.999999999999996
U spectral conditionality number:  17.62960473076362


### Test 2
#### Vandermonde matrix

\begin{bmatrix}
1 & x_0 & x_0^2 & \dots & x_0^n\\
1 & x_1 & x_1^2 & \dots & x_1^n\\
1 & x_2 & x_2^2 & \dots & x_2^n\\
\vdots & \vdots & \vdots & \ddots &\vdots \\
1 & x_m & x_m^2 & \dots & x_m^n
\end{bmatrix}

In [117]:
matrix_size = 5
a_matrix = np.vander(np.arange(1, matrix_size + 1), increasing=True)
print("A: ", a_matrix)
x_vector = np.array(np.random.rand(matrix_size))
b_vector = np.matmul(a_matrix, x_vector)
print("b: ", b_vector)

l_matrix, u_matrix = decompose_to_lu(a_matrix)
print("L: ")
print(l_matrix)
print("U: ")
print(u_matrix)
lu = np.matmul(l_matrix, u_matrix)
print("Allright! LU = A" if (lu == a_matrix).all() else "Error! LU != A")

print("A spectral conditionality number: ", calculate_spectral_conditionality_number(a_matrix))
print("L spectral conditionality number: ", calculate_spectral_conditionality_number(l_matrix))
print("U spectral conditionality number: ", calculate_spectral_conditionality_number(u_matrix))

result = solve_with_lu(l_matrix, u_matrix, b_vector)
print("Result: ")
print(result)
print("Result is correct!" if np.allclose(result, x_vector) else "Result is NOT correct!")

A:  [[  1   1   1   1   1]
 [  1   2   4   8  16]
 [  1   3   9  27  81]
 [  1   4  16  64 256]
 [  1   5  25 125 625]]
b:  [  2.65841781  18.08485646  78.11737952 232.94907979 552.06119253]
L: 
[[1. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 2. 1. 0. 0.]
 [1. 3. 3. 1. 0.]
 [1. 4. 6. 4. 1.]]
U: 
[[ 1.  1.  1.  1.  1.]
 [ 0.  1.  3.  7. 15.]
 [ 0.  0.  2. 12. 50.]
 [ 0.  0.  0.  6. 60.]
 [ 0.  0.  0.  0. 24.]]
Allright! LU = A
A spectral conditionality number:  26232.060161422698
L spectral conditionality number:  99.0
U spectral conditionality number:  373.66951474531606
Result: 
[0.93311323 0.35435205 0.23849048 0.32878945 0.8036726 ]
Result is correct!


### Test 3
#### Well-conditioned matrix

In [197]:
matrix_size = 5
a_matrix = generate_matrix_with_nonzero_major_minors(5)
while calculate_spectral_conditionality_number(a_matrix) > 10:
    a_matrix = generate_matrix_with_nonzero_major_minors(5)
print("A: \n", a_matrix)
x_vector = np.array(np.random.rand(matrix_size))
b_vector = np.matmul(a_matrix, x_vector)
print("b: ", b_vector)

l_matrix, u_matrix = decompose_to_lu(a_matrix)
print("L: ")
print(l_matrix)
print("U: ")
print(u_matrix)
lu = np.matmul(l_matrix, u_matrix)
print("Allright! LU = A" if np.allclose(lu, a_matrix) else "Error! LU != A")

print("A spectral conditionality number: ", calculate_spectral_conditionality_number(a_matrix))
print("L spectral conditionality number: ", calculate_spectral_conditionality_number(l_matrix))
print("U spectral conditionality number: ", calculate_spectral_conditionality_number(u_matrix))

result = solve_with_lu(l_matrix, u_matrix, b_vector)
print("Result: ")
print(result)
print("Result is correct!" if np.allclose(result, x_vector) else "Result is NOT correct!")

A: 
 [[0.01815358 0.87119148 0.38989192 0.38171046 0.86530286]
 [0.19139962 0.47903004 0.74032731 0.91228514 0.73378439]
 [0.42563081 0.42187261 0.03642776 0.54304968 0.075887  ]
 [0.10728464 0.37853158 0.83888271 0.07936674 0.10683945]
 [0.36506328 0.22829807 0.20042344 0.11509036 0.78373038]]
b:  [0.82022178 0.9184972  0.40014029 0.57474401 0.48907623]
L: 
[[ 1.          0.          0.          0.          0.        ]
 [10.54335137  1.          0.          0.          0.        ]
 [23.44610234  2.29768005  1.          0.          0.        ]
 [ 5.90983239  0.54788976 -0.28020981  1.          0.        ]
 [20.10970739  1.98605737  0.69538807  0.61561092  1.        ]]
U: 
[[ 0.01815358  0.87119148  0.38989192  0.38171046  0.86530286]
 [ 0.         -8.70624787 -3.37044023 -3.11222231 -8.38940768]
 [ 0.          0.         -1.36082488 -1.2556816  -0.93591769]
 [ 0.          0.          0.         -0.82317764 -0.67273816]
 [ 0.          0.          0.          0.          1.10955899]]
All